In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv("Dataset.csv")
print(df)


In [ ]:
performance_cols = [
    'kills',
    'deaths',
    'assists',
    'kda_ratio',
    'kill_participation',
    'gold_per_min',
    'damage_champ_per_min',
    'vision_score'
]

df_norm = df.copy()

for col in performance_cols:
    df_norm[col + '_z'] = (
        (df_norm[col] - df_norm[col].mean()) /
        df_norm[col].std()
    )
print(df_norm)

In [ ]:
z_cols = [c + '_z' for c in performance_cols]

df_perf_z = df_norm[z_cols + ['win']].dropna()

mean_perf_z = df_perf_z.groupby('win').mean()

mean_perf_z.T.plot(
    kind='bar',
    figsize=(12,6)
)
plt.axhline(0, color='black', linewidth=1, label='_nolegend_')
plt.title("Phong độ chuẩn hóa (Z-score) giữa thắng và thua")
plt.ylabel("Z-score")
plt.xticks(rotation=45)
plt.legend(["lose","win"])
plt.show()

In [ ]:
corr = df_perf_z.corr()['win'].sort_values(ascending=False)

plt.figure(figsize=(6,4))
sns.barplot(
    x=corr.values,
    y=corr.index
)
plt.title("Correlation giữa phong độ chuẩn hóa và chiến thắng")
plt.xlabel("Correlation coefficient")
plt.show()


In [ ]:
lanes = ['TOP', 'JUNGLE', 'MIDDLE', 'BOTTOM', 'SUPPORT']

df_lane = df_norm[
    df_norm['position'].isin(lanes)
][
    [
        'game_id',
        'position',
        'champion_name',
        'win',
        'gold_per_min',
        'kills_z',
        'deaths_z',
        'kda_ratio_z',
        'kill_participation_z',
        'damage_champ_per_min_z',
        'vision_score_z'
    ]
].dropna()
print(df_lane)

In [ ]:
lane_pairs = (
    df_lane
    .groupby(['game_id', 'position'])
    .filter(lambda x: len(x) == 2)
    .sort_values(['game_id', 'position'])
)

lane_pairs['opponent_gpm'] = (
    lane_pairs
    .groupby(['game_id', 'position'])['gold_per_min']
    .transform(lambda x: x[::-1].values)
)

lane_pairs['gpm_diff'] = (
    lane_pairs['gold_per_min'] - lane_pairs['opponent_gpm']
)
lane_pairs['lane_result'] = lane_pairs['gpm_diff'].apply(
    lambda x: 'Win lane' if x > 0 else 'Lose lane'
)

print(lane_pairs)

In [ ]:
lose_lane = lane_pairs[lane_pairs['lane_result'] == 'Lose lane']

lose_lane_win  = lose_lane[lose_lane['win'] == 1]
lose_lane_lose = lose_lane[lose_lane['win'] == 0]

compare_z_cols = [
    'deaths_z',
    'kill_participation_z',
    'vision_score_z',
    'damage_champ_per_min_z'
]

summary_z = pd.DataFrame({
    'Lose lane & Win game': lose_lane_win[compare_z_cols].mean(),
    'Lose lane & Lose game': lose_lane_lose[compare_z_cols].mean()
})

summary_z.round(2)
print(summary_z)

In [ ]:
summary_z.plot(
    kind='bar',
    figsize=(12,6)
)
plt.axhline(0, color='black', linewidth=1)
plt.title("Phong độ chuẩn hóa khi thua lane")
plt.ylabel("Z-score")
plt.xticks(rotation=45)
plt.show()


In [ ]:
plt.figure(figsize=(7,5))
sns.boxplot(
    data=lose_lane,
    x='win',
    y='deaths_z'
)
plt.xticks([0,1], ['Lose game', 'Win game'])
plt.title("Deaths (Z-score) khi thua lane")
plt.show()
